In [100]:
import json
from math import sin, cos, tan, pi
import numpy as np

In [11]:
with open('isd.isd', 'r') as f:
    isd = json.load(f)

The example that we have been working on looks like a pinhole ground to image projection, defined as:

$$\begin{bmatrix}
    w \cdot u \\
    w \cdot v \\
    w
\end{bmatrix} = \mathbf{K}
\begin{bmatrix}
    \mathbf{Rt}
\end{bmatrix}
\begin{bmatrix}
    X\\
    Y\\
    Z\\
    1
\end{bmatrix}
$$

or 

$$\begin{bmatrix}
    w \cdot u \\
    w \cdot v \\
    w
\end{bmatrix} =
\begin{bmatrix}
    f & s & u_{0} \\
    0 & \alpha f & v_{0} \\
    0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
    r_{11} & r_{12} & r_{13} & t_{x} \\
    r_{21} & r_{22} & r_{23} & t_{y} \\
    r_{31} & r_{32} & r_{33} & t_{z} \\
\end{bmatrix}
\begin{bmatrix}
    X\\
    Y\\
    Z\\
    1
\end{bmatrix}
$$

**K** is the intrinsic matrix (interior orientation), **R** is the extrinsic matrix (exterior orientation), and **t** is the translation.  In the extrinsic matrix $\alpha$ (pixel aspect ratio) and $s$ (skew) are often assume to be unit and zero, respectively.  $f$ is the focal length (in pixels) and ($u_{0}, v_{0}$) are the optical center (principal point).

The second resource below suggests that **t** can be thought of as the world origin in camera coordinates.

### Focal Length Conversion from mm to pixels
* If the sensor's physical width is known: $focal_{pixel} = (focal_{mm} / sensor_{width}) * imagewidth_{pixels}$
* If the horizontal FoV is known: $focal_{pixel} = (imagewidth_{pixels} * 0.5) / \tan(FoV * 0.5)$

### Resources:
* http://ksimek.github.io/2013/08/13/intrinsic/
* http://ksimek.github.io/2012/08/22/extrinsic/
* http://slazebni.cs.illinois.edu/spring16/3dscene_book_svg.pdf

In [152]:
# 512, 512 are the focal width/height in pixels divided by 2
def create_intrinsic_matrix(focal_length, image_width, sensor_width=14.4, skew=0, pixel_aspect=1):
    focal_pixels = (focal_length / sensor_width) * image_width  # From the IK - how do we get 14.4 automatically
    print(focal_pixels) # Scale back to mm
    intrinsic_matrix = np.zeros((3,3))
    intrinsic_matrix[0,0] = focal_pixels
    intrinsic_matrix[1,1] = focal_pixels
    intrinsic_matrix[:,2] = [512.5, 512.5, 1]
    return intrinsic_matrix

K = create_intrinsic_matrix(isd['focal_length'], isd['nsamples'])


39048.378278205884


In [151]:
def rotation_from_opk(o, p, k):
    om = np.empty((3,3))
    om[:,0] = [1,0,0]
    om[:,1] = [0, cos(o), -sin(o)]
    om[:,2] = [0, sin(o), cos(o)]
    
    pm = np.empty((3,3))
    pm[:,0] = [cos(p), 0, sin(p)]
    pm[:,1] = [0,1,0]
    pm[:,2] = [-sin(p), 0, cos(p)]
    
    km = np.empty((3,3))
    km[:,0] = [cos(k), -sin(k), 0]
    km[:,1] = [sin(k), cos(k), 0]
    km[:,2] = [0,0,1]
    
    return km.dot(pm).dot(om)

# This makes a great test case (Mikhail p.95 has the rotation matrix.)
o = isd['omega']
p = isd['phi']
k = isd['kappa']

# This is R, but we need t to have a proper augmented matrix
R = np.empty((3,4))
R[:,:3] = rotation_from_opk(o, p, k)

RC = np.empty((4,4))
RC[:3,:3] = rotation_from_opk(o, p, k)
RC[:3,-1] = [isd['x_sensor_origin'],
           isd['y_sensor_origin'],
           isd['z_sensor_origin']]
RC[-1] = [0,0,0,1]


invRC = np.linalg.inv(RC)[:3, :]
print(invRC)

def setfocalrot(x, y, z):    
    # This is a focal plan rotation matrix that is flipping the camera vertically (I think)
    # 0,0,1000 is the z position of the spacecraft
    c = np.zeros((3,4))
    c[0,0] = 1
    c[1,1] = -1
    c[2,2] = -1
    c[:,3] = [x,y,z]
    return c

# Arguments are spacecraft position: x, y, z
c = setfocalrot(isd['x_sensor_origin'],
                isd['y_sensor_origin'],
                isd['z_sensor_origin'])

def pixelloc(K,R,t, tx, ty):
    res = K.dot(R).dot(t)
    res[0] /= res[-1]
    res[1] /= res[-1]
    res[2] /= res[-1]
    # Mapping from focal plane to pixel space
    res[0]
    res[1]
    return res[:2]
    
# pixel position on the surface: x,y,z,1
position = np.array([1116890,
                     -1604470,
                     1459570,
                     1])

"""position = np.array([1131980,
                    -1597990,
                    1455060,
                    1])"""

ploc = pixelloc(K, invRC, position, isd['transx'][1], isd['transy'][2])
ploc

[[  5.68490235e-01   8.17280054e-01   9.41921758e-02   5.28070122e+05]
 [  5.61234754e-01  -3.01556882e-01  -7.70765203e-01   5.61822702e+03]
 [ -6.01526728e-01   4.91036414e-01  -6.30118113e-01   3.37759481e+06]]


array([  89.62335443,  172.51556848])

In [129]:
print(K)

[[  3.90483783e+04   0.00000000e+00   5.12500000e+02]
 [  0.00000000e+00   3.90483783e+04   5.12500000e+02]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00]]


In [130]:
print(R)

[[ 0.56849023  0.56123475 -0.60152673]
 [ 0.81728005 -0.30155688  0.49103641]
 [ 0.09419218 -0.7707652  -0.63011811]]
